In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)


In [5]:
df1 = pd.read_csv("C:\\Users\\tyler\\Downloads\\stock_data_july_2025.csv")
df1.head()

,Date,Ticker,Open Price,Close Price,High Price,Low Price,Volume Traded,Market Cap,PE Ratio,Dividend Yield,EPS,52 Week High,52 Week Low,Sector
0,2025-06-01,AAPL,185.96,187.79,188.85,183.74,42257183,2.931356e+12,28.37,0.49,6.62,248.20,155.68,Technology
1,2025-06-01,ABBV,189.63,193.64,194.31,188.03,33074970,3.289227e+11,16.74,3.51,11.57,210.83,127.91,Healthcare
2,2025-06-01,ABT,118.79,120.08,120.53,118.01,17130934,2.001333e+11,19.39,1.84,6.19,140.79,99.26,Healthcare
3,2025-06-01,ADBE,584.51,567.78,587.97,561.25,11080360,2.460380e+11,43.29,0.00,13.12,727.11,407.82,Technology
4,2025-06-01,ADP,253.46,255.60,258.59,249.91,10319559,1.052471e+11,29.92,2.04,8.54,303.75,234.28,Financials


In [7]:
df1.shape

(4346, 14)

In [14]:
df1.groupby('Sector')['Sector'].agg('count')

Sector
Communication Services    318
Consumer Discretionary    371
Consumer Staples          371
Energy                    106
Financials                795
Healthcare                954
Industrials               371
Real Estate               106
Technology                795
Utilities                 159
Name: Sector, dtype: int64

In [7]:
df2 = df1.drop(['Open Price', 'Low Price', 'High Price', 'Volume Traded'], axis = 'columns')
df2.head()

,Date,Ticker,Close Price,Market Cap,PE Ratio,Dividend Yield,EPS,52 Week High,52 Week Low,Sector
0,2025-06-01,AAPL,187.79,2.931356e+12,28.37,0.49,6.62,248.20,155.68,Technology
1,2025-06-01,ABBV,193.64,3.289227e+11,16.74,3.51,11.57,210.83,127.91,Healthcare
2,2025-06-01,ABT,120.08,2.001333e+11,19.39,1.84,6.19,140.79,99.26,Healthcare
3,2025-06-01,ADBE,567.78,2.460380e+11,43.29,0.00,13.12,727.11,407.82,Technology
4,2025-06-01,ADP,255.60,1.052471e+11,29.92,2.04,8.54,303.75,234.28,Financials


In [10]:
df3 = df2.copy()
df3["SectorAvgPE"] = df3.groupby("Sector")["PE Ratio"].transform("mean")
df3["PEScore"] = (df3["SectorAvgPE"] - df3["PE Ratio"]) / df3["SectorAvgPE"]
df3["PEScore"] = df3["PEScore"].clip(lower=0)
df3["PricePosition"] = (df3["Close Price"] - df3["52 Week Low"]) / (df3["52 Week High"] - df3["52 Week Low"])
df3["PriceScore"] = 1 - df3["PricePosition"]
df3["YieldScore"] = (df3["Dividend Yield"] / 5).clip(upper=1)
df3["UndervaluationScore"] = (0.4 * df3["PEScore"]) + (0.4 * df3["PriceScore"]) + (0.2 * df3["YieldScore"])
df3 = df3.sort_values("UndervaluationScore", ascending=False)
print(df3[["Date", "Ticker", "Sector", "UndervaluationScore"]].head(10))

            Date Ticker                  Sector  UndervaluationScore
4167  2025-07-21      T  Communication Services             0.789258
2527  2025-07-01      T  Communication Services             0.749991
2691  2025-07-03      T  Communication Services             0.746030
2199  2025-06-27      T  Communication Services             0.734499
2773  2025-07-04      T  Communication Services             0.730795
4003  2025-07-19      T  Communication Services             0.730715
3194  2025-07-09     VZ  Communication Services             0.730643
1800  2025-06-22     VZ  Communication Services             0.730317
1297  2025-06-16      T  Communication Services             0.730225
1953  2025-06-24      T  Communication Services             0.727949
